# GENERATE VIRTUAL IMAGES
In this notebook, we generate a set of virtual images of a BIM model from imageNodes
As ouput, the method generates a new set of virtual imageNodes

>This codebase operates on the scan2bim2.yml environment (python 3.8)

In [1]:
#IMPORT PACKAGES
from rdflib import Graph

import os.path, time
import importlib
import numpy as np
import open3d as o3d

#IMPORT MODULES
from context import geomapi 
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

## INITIALIZE SESSION

In [2]:
## INPUTS
projectPath= os.path.abspath(os.path.join(os.getcwd(), os.pardir))+"\\tests"#"D:\\Data\\2018-06 Werfopvolging Academiestraat Gent" 
sessionPath = projectPath + "\\Samples6" #"K:\Projects\2025-03 Project FWO SB Jelle\7.Data\21-11 House Maarten\RAW data\session_22-03-13 canon

idx=2 # index of the node you want to test

## READ IMAGE & BIM GRAPHS
A specifc NodeType (Mesh,PCD,IMG,BIM) is created per resource in the session

In [3]:
## read all graph files in a session and merge them
allSessionFilePaths=ut.get_list_of_files(sessionPath) 
rdfGraphPaths=[]

# collect all RDF graphs in the session
sessionGraph = Graph()
for path in allSessionFilePaths:        
    if path.endswith("imgGraph.ttl") and 'features' not in path:
        rdfGraphPaths.append(path)
        sessionGraph.parse(path)    
print('Total graph size = '+str(len(sessionGraph)) +' triples from '+ str(len(rdfGraphPaths))+' paths')
imgNodelist=tl.graph_to_nodes(sessionGraph,sessionPath=sessionPath)
print (str(len(imgNodelist))+' Nodes succesfully created from the session!') 

Total graph size = 513 triples from 1 paths
27 Nodes succesfully created from the session!


In [4]:
# READ MESH
from geomapi.meshnode import MeshNode
mesh = o3d.io.read_triangle_mesh("D:\\Scan-to-BIM repository\\geomapi\\tests\\Samples5\\week22.obj")
meshNode=MeshNode(mesh=mesh,sessionPath=sessionPath)
print(meshNode.mesh)

TriangleMesh with 330263 points and 485077 triangles.


In [5]:
## read all graph files in a session and merge them
allSessionFilePaths=ut.get_list_of_files(sessionPath) 
rdfGraphPaths=[]

# collect all RDF graphs in the session
sessionGraph = Graph()
for path in allSessionFilePaths:        
    if path.endswith("bimGraph.ttl") and 'features' not in path:
        rdfGraphPaths.append(path)
        sessionGraph.parse(path)    
print('Total graph size = '+str(len(sessionGraph)) +' triples from '+ str(len(rdfGraphPaths))+' paths')
bimNodelist=tl.graph_to_nodes(sessionGraph,sessionPath=sessionPath,getGeometry=True)
print (str(len(bimNodelist))+' Nodes succesfully created from the session!') 

Total graph size = 14872 triples from 1 paths
976 Nodes succesfully created from the session!


In [ ]:
importlib.reload(ld)
importlib.reload(gt1)
importlib.reload(gt2)

importlib.reload(ut)
importlib.reload(geomapi.node)
importlib.reload(geomapi.imagenode)
importlib.reload(geomapi.bimnode)
importlib.reload(geomapi.meshnode)

NameError: name 'ld' is not defined

## GENERATE 1 VIRTUAL IMAGE of a mesh

In [ ]:
imnode=imgNodelist[23]
imnode.pinholeCamera=imnode.get_pinhole_camera_parameters()
img=gt1.generate_virtual_image(meshNode.mesh,imnode.pinholeCamera)
print(img)

Image of size 1404x936, with 3 channels.
Use numpy.asarray to access buffer data.


## GENERATE 1 VIRTUAL IMAGE PER NODE of a mesh (slow)

In [ ]:
for node in imgNodelist: 
    img=node.get_virtual_image(meshNode.mesh)
    o3d.io.write_image(sessionPath+"\\IMG_V\\"+node.name+".png", img)

## GENERATE A SET OF VIRTUAL IMAGES with same settings of a mesh (6xfaster)

In [ ]:
cartesianTransforms=[node.cartesianTransform for node in imgNodelist if getattr(node,'cartesianTransform',None) is not None ]
imgList=gt1.generate_virtual_images(meshNode.mesh,cartesianTransforms,width=640,height=480,f=400) 

for idx,img in enumerate(imgList):
    o3d.io.write_image(sessionPath+"\\IMG_V\\"+str(idx)+".png", img)

## GENERATE 1 VIRTUAL IMAGE of a point cloud (NOT TESTED)

In [ ]:
# READ PCD
from geomapi.pointcloudnode import PointCloudNode
pcd = o3d.io.read_point_cloud("D:\\Scan-to-BIM repository\\geomapi\\tests\\Samples4\\academiestraat week 22 19.pcd")
pcdNode=PointCloudNode(pcd=pcd,sessionPath=sessionPath, getGeometry=True)
print(pcdNode.pcd)

PointCloud with 11934055 points.


## GENERATE VIRTUAL IMAGE OF BIM OBJECT

In [6]:
# simple wall
myNode=bimNodelist[0]
o3d.visualization.draw_geometries([myNode.mesh])

In [9]:
# get centroid
import math
from scipy.spatial.transform import Rotation as R
import copy
fov=math.pi / 3#60 #degrees

#determine extrinsic camera parameters
array=np.empty((1,3),dtype=float)
if getattr(myNode,"orientedBounds",None) is not None:
    box=myNode.get_bounding_box()
    c=box.get_center()
    u=box.extent[0]
    print(u)
    d_w=math.cos(fov/2)*u
    print(str(d_w))

    #determine c_i 
    rotation_matrix=box.R
    pcd = o3d.geometry.PointCloud()
    array=np.array([[c[0],c[1],c[2]+d_w]])
    pcd.points = o3d.utility.Vector3dVector(array)
    pcd.rotate(rotation_matrix, center =c) 
    c_i=np.asarray(pcd.points[0])

#generate scene
width=640
height=480
render = o3d.visualization.rendering.OffscreenRenderer(width,height)
mtl=o3d.visualization.rendering.MaterialRecord()
mtl.base_color = [1.0, 1.0, 1.0, 1.0]  # RGBA
mtl.shader = "defaultUnlit"

#set camera
# Look at the origin from the front (along the -Z direction, into the screen), with Y as Up.
center = c  # look_at target
eye = c_i  # camera position
up = [0, 0, 1]  # camera orientation
render.scene.camera.look_at(center, eye, up)

#add geometries
myNode.mesh.paint_uniform_color([1,0,0])
render.scene.add_geometry("test",myNode.mesh,mtl) 

#render the image
img = render.render_to_image()
o3d.io.write_image(sessionPath+"\\IMG_V\\"+"test"+".png", img)

print(c)
print(c_i)
print(d_w)
testbox1= copy.deepcopy(box)
testbox1.rotate(rotation_matrix.transpose())

testbox2= copy.deepcopy(testbox1)
testbox2.rotate(rotation_matrix)

testbox2.translate([1,0,0])
box.color=[1,0,0]
testbox1.color=[0,1,0]
testbox2.color=[0,0,1]


1.420000003141028
1.229756076094113
[-11.62643457  73.90228863   9.052     ]
[-11.59700347  75.13169248   9.052     ]
1.229756076094113


In [10]:
o3d.visualization.draw_geometries([myNode.mesh,pcd,box,testbox1,testbox2])

In [11]:
# this is resulting in a blank!
imnode=imgNodelist[23]
imnode.pinholeCamera=imnode.get_pinhole_camera_parameters()
img=gt1.generate_virtual_image(pcdNode.pcd,imnode.pinholeCamera)
o3d.io.write_image(sessionPath+"\\IMG_V\\"+"myPointCloudImage"+".png", img)

width=pinholeCamera.intrinsic.width
height=pinholeCamera.intrinsic.height
render = o3d.visualization.rendering.OffscreenRenderer(width,height)

# Define a simple unlit Material. (The base color does not replace the geometry colors)
mtl=o3d.visualization.rendering.MaterialRecord()
mtl.base_color = [1.0, 1.0, 1.0, 1.0]  # RGBA
mtl.shader = "defaultUnlit"

#set camera
render.setup_camera(pinholeCamera.intrinsic,pinholeCamera.extrinsic)

#add geometries
geometries=ut.item_to_list(geometries)

for idx,geometry in enumerate(geometries):
    render.scene.add_geometry(str(idx),geometry,mtl) 

#render image
img = render.render_to_image()

True